## Prepare Data

In [1]:
import re
import os
import pandas as pd

regex = re.compile(r'\[.+\]\n', re.IGNORECASE)
file_paths, file_names, sessions, emotions = [], [], [], []
emotion_map = {'neu': 'neutral', 'ang': 'angry', 'hap': 'happy', 'exc': 'happy', 'sad': 'sad'}

for session in range(1, 6):
    emo_evaluation_dir = f'/home/jz3313/IEMOCAP_full_release/Session{session}/dialog/EmoEvaluation/'
    file_dir = f'/home/jz3313/IEMOCAP_full_release/Session{session}/sentences/wav/'
    evaluation_files = [l for l in os.listdir(emo_evaluation_dir) if 'Ses' in l]
    for file in evaluation_files:
        with open(emo_evaluation_dir + file) as f:
            content = f.read()
        lines = re.findall(regex, content)
        for line in lines[1:]:  # the first line is a header
            start_end_time, wav_file_name, emotion, val_act_dom = line.strip().split('\t')
            dir_name = '_'.join(wav_file_name.split('_')[:-1])
            if emotion in emotion_map:
                file_paths.append(file_dir+dir_name+'/'+wav_file_name+'.wav')
                file_names.append(wav_file_name)
                sessions.append(session)
                emotions.append(emotion_map[emotion])

In [2]:
file = pd.DataFrame({'path':file_paths, 'name': file_names, 'session': sessions, 'emotion': emotions})

In [3]:
file.head()

,path,name,session,emotion
0,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F000,1,sad
1,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F001,1,sad
2,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F002,1,sad
3,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F003,1,neutral
4,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F004,1,sad


In [4]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)

torch.random.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

1.12.1+cu113
0.12.1+cu113
cuda


In [5]:
bundle = torchaudio.pipelines.HUBERT_BASE
extractor = bundle.get_model()
print(extractor.__class__)

<class 'torchaudio.models.wav2vec2.model.Wav2Vec2Model'>


In [ ]:
from tqdm import tqdm

for _, row in tqdm(file.iterrows()):
    path, name, session, emotion = row[0], row[1], row[2], row[3]
    wave, sr = torchaudio.load(path)
    if sr != bundle.sample_rate:
        wave = torchaudio.functional.resample(wave, sr, bundle.sample_rate)
    with torch.inference_mode():
        feature, _ = extractor.extract_features(wave)
    feature = [f[0] for f in feature]
    feature = torch.stack(feature)
    save_path = f'../data/hubertbase/Session{session}/{name}.pt'
    torch.save(feature, save_path)

## Load Data

In [6]:
file['newpath'] = file.apply(lambda x: f'../data/hubertbase/Session{x[2]}/{x[1]}.pt', axis=1)

In [7]:
file.head()

,path,name,session,emotion,newpath
0,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F000,1,sad,../data/hubertbase/Session1/Ses01F_impro02_F00...
1,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F001,1,sad,../data/hubertbase/Session1/Ses01F_impro02_F00...
2,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F002,1,sad,../data/hubertbase/Session1/Ses01F_impro02_F00...
3,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F003,1,neutral,../data/hubertbase/Session1/Ses01F_impro02_F00...
4,/home/jz3313/IEMOCAP_full_release/Session1/sen...,Ses01F_impro02_F004,1,sad,../data/hubertbase/Session1/Ses01F_impro02_F00...


In [8]:
holdout = 5
train = file[file['session'] != holdout]
test = file[file['session'] == holdout]

In [9]:
class MyDataSet(torch.utils.data.Dataset):
    def __init__(self, datas, labels, label_transform):
        super(MyDataSet).__init__()
        self.datas = datas
        self.labels = labels
        self.label_transform = label_transform
        
    def __getitem__(self, idx):
        audio = self.datas[idx]
        label = self.label_transform[self.labels[idx]]
        length = audio.size(1)
        return audio, length, label
    
    def __len__(self):
        return len(self.labels)

In [10]:
def collate_indic(data):
    audios, lengths, labels = zip(*data)
    max_len = max(lengths)
    n_ftrs = audios[0].size(2)
    n_dims = audios[0].size(0)
    features = torch.zeros((len(audios), n_dims, max_len, n_ftrs))
    labels = torch.tensor(labels)
    lengths = torch.tensor(lengths)

    for i in range(len(data)):
        j, k = audios[i].size(1), audios[i].size(2)
        features[i] = torch.cat([audios[i], torch.zeros((n_dims, max_len - j, k))], dim=1)

    return features, lengths, labels

In [11]:
categories = ['angry', 'happy', 'neutral', 'sad']
cate_dic = {}
for i, cate in enumerate(categories):
    cate_dic[cate] = i
cate_dic

{'angry': 0, 'happy': 1, 'neutral': 2, 'sad': 3}

## Train with 3CNN+LSTM

In [12]:
import torch.nn as nn
import torch.nn.functional as F

class ICASSP3CNN(nn.Module):
    def __init__(self, vocab_size, dims = 12, embed_size=128, hidden_size=512, num_lstm_layers = 2, bidirectional = False, label_size=7):
        super().__init__()
        self.n_layers = num_lstm_layers 
        self.hidden = hidden_size
        self.bidirectional = bidirectional
        
        self.aggr = nn.Conv1d(in_channels=dims, out_channels=1, kernel_size=1)
        
        self.embed = nn.Linear(in_features = vocab_size, out_features = embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(3 * embed_size)

        self.lstm = nn.LSTM(input_size = 3 * embed_size, 
                            hidden_size = hidden_size, 
                            num_layers = num_lstm_layers, 
                            bidirectional = bidirectional)

        self.linear = nn.Linear(in_features = 2 * hidden_size if bidirectional else hidden_size, 
                                out_features = label_size)


    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """
        n, d, b, t = x.size(0), x.size(1), x.size(2), x.size(3)
        x = torch.flatten(x, start_dim=2)
        input = self.aggr(x)
        input = torch.reshape(input, (n, b, t))
        input = self.embed(input)

        batch_size = input.size(0)
        input = input.transpose(1,2)    # (B,T,H) -> (B,H,T)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True, enforce_sorted=False)
        _, (hn, cn) = self.lstm(pack_tensor)

        if self.bidirectional:
            h_n = hn.view(self.n_layers, 2, batch_size, self.hidden)
            h_n = torch.cat([ h_n[-1, 0,:], h_n[-1,1,:] ], dim = 1)
        else:
            h_n = hn[-1]

        logits = self.linear(h_n)

        return logits

### Model Traning on each layer 

In [13]:
from tqdm import tqdm
from torch.utils.data import DataLoader

traindata = []
for _, row in tqdm(train.iterrows()):
    traindata.append(torch.load(row[4]))

train_dataset = MyDataSet(traindata, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=64, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

4290it [07:49,  9.14it/s]


In [14]:
train_dataset = MyDataSet(traindata, train['emotion'].tolist(), cate_dic)
trainloader_args = dict(batch_size=64, shuffle=True)
train_dataloader = DataLoader(train_dataset, **trainloader_args, 
                              collate_fn=collate_indic)

In [16]:
from tqdm import tqdm
from torchsummary import summary
import torch.optim as optim

model = ICASSP3CNN(768)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
epochs = 50
train_losses = []
train_accuracies = []
valid_losses = []
valid_accuracies = []

for epoch in tqdm(range(epochs)):
    train_loss = 0
    acc_cnt = 0
    err_cnt = 0
    batch_cnt = 0
    model.train()
    for batch, (x, length, y) in enumerate(train_dataloader):
        x = x.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        logits = model(x, length)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().item()

        #model outputs
        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
        batch_cnt += 1
    
    train_loss = train_loss/batch_cnt
    train_accuracy = acc_cnt/(acc_cnt+err_cnt)
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    print(f"epoch:{epoch+1}, train accu:{train_accuracy:.4f},", f"train loss:{train_loss:.2f}")

  2%|███▎                                                                                                                                                                  | 1/50 [01:34<1:17:17, 94.65s/it]

epoch:1, train accu:0.4748, train loss:1.15


  4%|██████▋                                                                                                                                                               | 2/50 [03:07<1:14:47, 93.49s/it]

epoch:2, train accu:0.5583, train loss:0.99


  6%|█████████▉                                                                                                                                                            | 3/50 [04:39<1:12:46, 92.89s/it]

...

epoch:48, train accu:0.8140, train loss:0.51


 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋   | 49/50 [1:15:00<01:31, 91.88s/it]

epoch:49, train accu:0.9009, train loss:0.32


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:16:31<00:00, 91.82s/it]

epoch:50, train accu:0.8930, train loss:0.32


## Model Test

In [18]:
from tqdm import tqdm

testdata = []
for _, row in tqdm(test.iterrows()):
    testdata.append(torch.load(row[4]))
    
test_dataset = MyDataSet(testdata, test['emotion'].tolist(), cate_dic)
testloader_args = dict(batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, **testloader_args, 
                             collate_fn=collate_indic)

1241it [02:07,  9.70it/s]


In [19]:
test_loss = 0
acc_cnt = 0
err_cnt = 0
batch_cnt = 0
model.eval()

for x, lengths, y in test_dataloader:

    x = x.to(device)
    y = y.to(device)

    logits = model(x, lengths)
    loss = criterion(logits, y)
    test_loss += loss.cpu().item()

    out_val, out_indices = torch.max(logits, dim=1)
    tar_indices = y

    for i in range(len(out_indices)):
        if out_indices[i] == tar_indices[i]:
            acc_cnt += 1
        else:
            err_cnt += 1
    batch_cnt += 1

test_loss = test_loss/batch_cnt
test_accuracy = acc_cnt/(acc_cnt+err_cnt)
print(f'test accuracy: {test_accuracy}')

test accuracy: 0.5898468976631749


In [20]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)

aggr.weight tensor([[[-0.1250],
         [ 0.1303],
         [ 0.2537],
         [-0.0435],
         [ 0.1299],
         [-0.2512],
         [-0.2828],
         [ 0.1415],
         [-0.0142],
         [ 0.0112],
         [-0.0022],
         [ 0.0660]]], device='cuda:0')
aggr.bias tensor([-0.0235], device='cuda:0')
embed.weight tensor([[ 0.0071,  0.1413, -0.0656,  ...,  0.0079, -0.1076,  0.0459],
        [ 0.0426, -0.0723,  0.0207,  ...,  0.0307, -0.0739, -0.1503],
        [ 0.0966,  0.0378,  0.0385,  ..., -0.0143, -0.0220, -0.0978],
        ...,
        [ 0.0796,  0.0220,  0.0242,  ..., -0.0295, -0.0784,  0.0260],
        [-0.0224,  0.0989,  0.0033,  ..., -0.0097, -0.1146, -0.0145],
        [-0.0209,  0.0935,  0.0756,  ..., -0.1003,  0.0828,  0.0328]],
       device='cuda:0')
embed.bias tensor([-3.1782e-02,  2.5740e-03, -6.0295e-02, -2.3978e-02, -4.5746e-03,
        -2.6784e-02, -4.2904e-02, -4.8631e-02,  3.1392e-02, -3.7244e-02,
         4.1521e-02, -1.0550e-02, -1.0406e-01, -4.4008e-

In [21]:
model_path = f'../models/hubertbase/holdout{holdout}.pth'

torch.save({'epoch':epochs,
            'model_state_dict':model.state_dict(),
            'optimizer_state_dict':optimizer.state_dict()},
            model_path)